In [1]:
from cogdl.datasets.grb_data import Cora_GRBDataset
from cogdl.utils import set_random_seed
from cogdl.utils.grb_utils import evaluate, GCNAdjNorm
import copy
import torch
dataset = Cora_GRBDataset()
graph = copy.deepcopy(dataset.get(0))
# device = "cpu"
device = "cuda:0"
device_ids = [0]
graph.to(device)
test_mask = graph.test_mask
set_random_seed(40)

/home/cenyukuo/anaconda3/envs/py3.7/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train surrogate model
from cogdl.models.nn import GCN
from cogdl.trainer import Trainer
from cogdl.wrappers import fetch_model_wrapper, fetch_data_wrapper
model_sur = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=2,
    dropout=0.5,
    activation="relu"
)
print(model_sur)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_sur, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_sur.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_sur.to(device)
test_score = evaluate(model_sur,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for surrogate model: {:.4f}.".format(test_score))

GCN(
  (layers): ModuleList(
    (0): GCNLayer(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (act): ReLU(inplace=True)
    )
    (1): GCNLayer(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)
Model Parameters: 19847


Epoch: 200, train_loss:  0.6122, val_acc:  0.7724: 100%|██████████| 200/200 [00:03<00:00, 51.84it/s] 


Saving 166-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.8022388059701493, 'val_acc': 0.832089552238806}
Test score before attack for surrogate model: 0.8022.


In [3]:
# train target model

model_target = GCN(
    in_feats=graph.num_features,
    hidden_size=64,
    out_feats=graph.num_classes,
    num_layers=3,
    dropout=0.5,
    activation=None,
    norm="layernorm"
)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score = evaluate(model_target,
                      graph,
                      mask=test_mask,
                      device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score))

Model Parameters: 24263


Epoch: 140, train_loss:  0.3691, val_acc:  0.7910:  70%|███████   | 140/200 [00:01<00:00, 80.70it/s]


Saving 91-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.8159203980099502, 'val_acc': 0.8059701492537313}
Test score before attack for target model: 0.8159.


In [4]:
# train target model

from cogdl.models.nn import SAGE

model_target = SAGE(
    graph.num_features,
    graph.num_classes,
    hidden_size=128,
    num_layers=2,
    dropout=0.5,
    norm=None
)
print(model_target)
mw_class = fetch_model_wrapper("node_classification_mw")
dw_class = fetch_data_wrapper("node_classification_dw")
optimizer_cfg = dict(
                    lr=0.01,
                    weight_decay=0
                )
model_wrapper = mw_class(model_target, optimizer_cfg)
dataset_wrapper = dw_class(dataset)
trainer = Trainer(epochs=200,
                  early_stopping=True,
                  patience=50,
                  cpu=device=="cpu",
                  device_ids=device_ids)
trainer.run(model_wrapper, dataset_wrapper)
model_target.load_state_dict(torch.load("./checkpoints/model.pt"), False)
model_target.to(device)
test_score_target_raw = evaluate(model_target,
                                 graph,
                                 mask=test_mask,
                                 device=device)
print("Test score before attack for target model: {:.4f}.".format(test_score_target_raw))

SAGE(
  (layers): ModuleList(
    (0): SAGELayer(
      (fc): Linear(in_features=604, out_features=128, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): SAGELayer(
      (fc): Linear(in_features=256, out_features=7, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
)
Model Parameters: 79239


Epoch: 140, train_loss:  1.8881, val_acc:  0.7612:  70%|███████   | 140/200 [00:01<00:00, 117.19it/s]


Saving 91-th model to ./checkpoints/model.pt ...
Loading model from ./checkpoints/model.pt ...
{'test_acc': 0.7972636815920398, 'val_acc': 0.8246268656716418}
Test score before attack for target model: 0.7973.


In [5]:
# FGSM attack
from cogdl.attack.injection import FGSM
attack = FGSM(epsilon=0.01,
              n_epoch=1000,
              n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: 2.1066, Surrogate test score: 0.0970: 100%|██████████| 1000/1000 [01:36<00:00, 10.35it/s]


Surrogate test score: 0.0970
Attack runtime: 96.9449.
Test score after attack for surrogate model: 0.1493.
Test score after attack for target model: 0.1194.


In [6]:
from cogdl.attack.injection import PGD
attack = PGD(epsilon=0.01,
              n_epoch=1000,
              n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: 2.1091, Surrogate test score: 0.0970: 100%|██████████| 1000/1000 [01:27<00:00, 11.46it/s]


Surrogate test score: 0.0970
Attack runtime: 87.5898.
Test score after attack for surrogate model: 0.1953.
Test score after attack for target model: 0.1194.


In [7]:
from cogdl.attack.injection import RAND
attack = RAND(n_inject_max=100,
              n_edge_max=200,
              feat_lim_min=-1,
              feat_lim_max=1,
              device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Loss: -1.8408, Surrogate test acc: 0.5373
Attack runtime: 0.3861.
Test score after attack for surrogate model: 0.2575.
Test score after attack for target model: 0.3085.


In [8]:
from cogdl.attack.injection import SPEIT
attack = SPEIT(lr=0.02,
               n_epoch=1000,
               n_inject_max=100,
               n_edge_max=200,
               feat_lim_min=-1,
               feat_lim_max=1,
               device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Epoch 999, Loss: -2.1066, Surrogate test score: 0.0970: 100%|██████████| 1000/1000 [01:39<00:00, 10.04it/s]


Surrogate test score: 0.0970
Attack runtime: 100.1495.
Test score after attack for surrogate model: 0.1692.
Test score after attack for target model: 0.1194.


In [9]:
from cogdl.attack.injection import TDGIA
attack = TDGIA(lr=0.01,
               n_epoch=1000,
               n_inject_max=100,
               n_edge_max=200,
               feat_lim_min=-1,
               feat_lim_max=1,
               device=device)
graph_attack = attack.attack(model=model_sur,
                             graph=graph,
                             adj_norm_func=GCNAdjNorm)

# apply injection attack
test_score_sur = evaluate(model_sur,
                          graph_attack,
                          mask=test_mask,
                          device=device)
print(graph_attack)
print("Test score after attack for surrogate model: {:.4f}.".format(test_score_sur))

# transfer to target model
test_score_target_attack = evaluate(model_target,
                                    graph_attack,
                                    mask=test_mask,
                                    device=device)
print("Test score after attack for target model: {:.4f}.".format(test_score_target_attack))

Attacking: Sequential inject 20/100 nodes


Epoch 999, Loss: 4.8468, Surrogate test score: 0.4391: 100%|██████████| 1000/1000 [01:01<00:00, 16.17it/s]


Attacking: Sequential inject 40/100 nodes


Epoch 999, Loss: 4.2344, Surrogate test score: 0.1853: 100%|██████████| 1000/1000 [01:00<00:00, 16.49it/s]


Attacking: Sequential inject 60/100 nodes


...

Attacking: Sequential inject 80/100 nodes


Epoch 999, Loss: 3.8405, Surrogate test score: 0.1182: 100%|██████████| 1000/1000 [01:29<00:00, 11.14it/s]


Attacking: Sequential inject 100/100 nodes


Epoch 999, Loss: 3.7608, Surrogate test score: 0.1095: 100%|██████████| 1000/1000 [01:22<00:00, 12.14it/s]


Attack runtime: 368.9408.
Graph(x=[2780, 302], y=[2680], edge_index=[2, 18336], edge_weight=[18336], edge_attr=[18336])
Test score after attack for surrogate model: 0.1729.
Test score after attack for target model: 0.0995.
